# 🛍️ ImpactStore Profit Prediction Challenge – Final Submission

This notebook presents a **complete and beginner-friendly pipeline** for the ImpactStore Hackathon — a data science competition to predict store-level product returns using real transactional retail data.

---

## 📚 Table of Contents

1. 🗂️ [Overview of the Dataset](#overview)
2. 📦 [Data Imports & Setup](#imports)
3. 👀 [Initial Data Exploration](#exploration)
4. ❓ [Missing Value Analysis](#missing-values)
5. 🧹 [Data Cleaning & Preprocessing](#cleaning)
6. 🧠 [Feature Engineering](#features)
7. 🧪 [Model Building: Base Models](#base-models)
8. 🔗 [Stacking Meta-Model](#stacking)
9. 📈 [Model Evaluation](#evaluation)
10. 💾 [Submission Generation](#submission)


## 📦 Data Imports & Setup
We install required libraries, import our tools, and load the datasets.

Installing necesary libraries

In [17]:
# 📦 Install dependencies
!pip install xgboost lightgbm catboost scikit-learn pandas --quiet
# 📚 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet, RidgeCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from google.colab import files
import warnings
warnings.filterwarnings("ignore")

Loading our data into the notebook:

*I imported my test.csv, train.csv and the sapmle submission then i loaded it*

In [18]:
# 📊 Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("SampleSubmission.csv")


## 📦 Initial data Exploration

Quickly understand what the dataset looks like — its shape, columns, and general structure — before diving into modeling.


In [19]:
train.head()

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
0,DRA12,BABATUNJI010,DRA12_BABATUNJI010,11.6,Low Sugar,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
1,DRA12,BABATUNJI013,DRA12_BABATUNJI013,11.6,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
2,DRA12,BABATUNJI017,DRA12_BABATUNJI017,11.6,Low Sugar,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
3,DRA12,BABATUNJI018,DRA12_BABATUNJI018,11.6,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
4,DRA12,BABATUNJI035,DRA12_BABATUNJI035,11.6,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77


In [20]:
test.head()

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type
0,DRA59,BABATUNJI010,DRA59_BABATUNJI010,8.270,Normal Sugar,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store
1,DRA59,BABATUNJI013,DRA59_BABATUNJI013,8.270,Normal Sugar,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1
2,DRB01,BABATUNJI013,DRB01_BABATUNJI013,7.390,Low Sugar,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1
3,DRB13,BABATUNJI010,DRB13_BABATUNJI010,6.115,Normal Sugar,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store
4,DRB13,BABATUNJI013,DRB13_BABATUNJI013,6.115,Normal Sugar,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1


In [21]:
sample.head()

,Item_Store_ID,Item_Store_Returns
0,DRA59_BABATUNJI010,100
1,DRA59_BABATUNJI013,100
2,DRB01_BABATUNJI013,100
3,DRB13_BABATUNJI010,100
4,DRB13_BABATUNJI013,100


In [22]:
train.shape

(4990, 13)

In [23]:
train.columns

Index(['Item_ID', 'Store_ID', 'Item_Store_ID', 'Item_Weight',
       'Item_Sugar_Content', 'Item_Visibility', 'Item_Type', 'Item_Price',
       'Store_Start_Year', 'Store_Size', 'Store_Location_Type', 'Store_Type',
       'Item_Store_Returns'],
      dtype='object')

In [24]:
train.describe()

,Item_Weight,Item_Visibility,Item_Price,Store_Start_Year,Item_Store_Returns
count,4188.000000,4990.000000,4990.000000,4990.000000,4990.000000
mean,12.908838,0.066916,391.803796,2004.783567,6103.520164
std,4.703256,0.053058,119.378259,8.283151,4447.333835
min,4.555000,0.000000,78.730000,1992.000000,83.230000
25%,8.767500,0.027273,307.890000,1994.000000,2757.660000
50%,12.600000,0.053564,393.860000,2006.000000,5374.675000
75%,17.100000,0.095358,465.067500,2011.000000,8522.240000
max,21.350000,0.328391,667.220000,2016.000000,32717.410000


## ❓ Missing Value Analysis
Identify which columns contain missing values and how many are missing, so you can decide how to clean them later.

In [25]:
train.isnull().sum()

,0
Item_ID,0
Store_ID,0
Item_Store_ID,0
Item_Weight,802
Item_Sugar_Content,0
Item_Visibility,0
Item_Type,0
Item_Price,0
Store_Start_Year,0
Store_Size,1450


In [26]:
test.isnull().sum()

,0
Item_ID,0
Store_ID,0
Item_Store_ID,0
Item_Weight,661
Item_Sugar_Content,0
Item_Visibility,0
Item_Type,0
Item_Price,0
Store_Start_Year,0
Store_Size,959


## 🧪 Data Cleaning and preprocessing
Fix missing or inconsistent data to make the dataset suitable for machine learning. This ensures high-quality input for the model.

In [45]:
# ✅ STEP 1: Fix Item_Weight missing values
# Fill by Item_ID group mean first
data['Item_Weight'] = data.groupby('Item_ID')['Item_Weight'].transform(
    lambda x: x.fillna(x.mean())
)
# Fallback to global mean
data['Item_Weight'].fillna(data['Item_Weight'].mean(), inplace=True)

# ✅ STEP 2: Fix Store_Size missing values
# Fill using most frequent value per Store_Type
def fill_store_mode(series):
    mode = series.mode()
    return series.fillna(mode[0] if not mode.empty else "Medium")

data['Store_Size'] = data.groupby('Store_Type')['Store_Size'].transform(fill_store_mode)
data['Store_Size'].fillna(data['Store_Size'].mode()[0], inplace=True)

# ✅ STEP 3: Recompute derived features that use imputed columns
data['Price_per_kg'] = data['Item_Price'] / (data['Item_Weight'] + 1)
data['Visibility_Weight_Ratio'] = data['Item_Visibility'] / (data['Item_Weight'] + 1)
data['Price_Weight'] = data['Item_Price'] * data['Item_Weight']

# ✅ STEP 4: Remove duplicate columns (from merges)
cols_to_drop = [col for col in data.columns if "_x" in col or "_y" in col]
data.drop(columns=cols_to_drop, inplace=True)

# ✅ Final check and fill any leftover numeric NaNs just in case
data.fillna(0, inplace=True)


Now lets check again

In [46]:
train.isnull().sum()

,0
Item_ID,0
Store_ID,0
Item_Store_ID,0
Item_Weight,0
Item_Sugar_Content,0
Item_Visibility,0
Item_Type,0
Item_Price,0
Store_Start_Year,0
Store_Size,0


Lets do   some prepocessing

In [29]:
# 🧹 Combine for feature processing
train['is_train'] = 1
test['is_train'] = 0
test['Item_Store_Returns'] = np.nan
data = pd.concat([train, test], ignore_index=True)


## 🧠 Feature Engineering
We extract meaningful features, encode categoricals, and apply transformations to improve model performance.

In [30]:
# 🧠 Feature Engineering
data['Store_Age'] = 2025 - data['Store_Start_Year']
data['Price_per_kg'] = data['Item_Price'] / data['Item_Weight']
data['Visibility_Weight_Ratio'] = data['Item_Visibility'] / (data['Item_Weight'] + 1)
data['Item_Sugar_Content'] = data['Item_Sugar_Content'].str.lower().str.strip()

In [33]:
# Interaction Features
data['Price_Vis'] = data['Item_Price'] * data['Item_Visibility']
data['Sugar_Vis'] = LabelEncoder().fit_transform(data['Item_Sugar_Content']) * data['Item_Visibility']
data['Price_Weight'] = data['Item_Price'] * data['Item_Weight']

In [34]:
# Group statistics
group_stats = data.groupby(['Item_Type'])['Item_Price'].agg(['mean', 'std']).reset_index()
group_stats.columns = ['Item_Type', 'Type_Price_Mean', 'Type_Price_Std']
data = data.merge(group_stats, on='Item_Type', how='left')


In [35]:
# 🚫 Skew correction
for col in ['Item_Visibility', 'Price_per_kg', 'Visibility_Weight_Ratio', 'Price_Vis']:
    data[col] = np.log1p(data[col])


In [36]:
# 🔢 Encode categoricals
cat_cols = ['Item_ID', 'Store_ID', 'Item_Sugar_Content', 'Item_Type', 'Store_Size', 'Store_Location_Type', 'Store_Type']
for col in cat_cols:
    data[col] = LabelEncoder().fit_transform(data[col])

## 📈  🧪 Model Building: Base Models
Train several different ML models (XGBoost, LightGBM, CatBoost, ExtraTrees) and generate predictions using cross-validation.


In [44]:
# ✂️ Split back
train = data[data['is_train'] == 1].copy()
test = data[data['is_train'] == 0].copy()
features = [col for col in train.columns if col not in ['Item_Store_Returns', 'Item_Store_ID', 'is_train']]

# 🎯 Target Transform with Capping
y_raw = train['Item_Store_Returns'].clip(0, 18000)  # cap outliers
y = np.log1p(y_raw)
X = train[features]
X_test = test[features]

# Stratified KFold based on binned target
bins = pd.qcut(y_raw, q=5, labels=False)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 💼 Define Base Models
base_models = {
    "xgb": XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=7, subsample=0.8, colsample_bytree=0.8, random_state=42),
    "lgb": lgb.LGBMRegressor(n_estimators=450, learning_rate=0.03, num_leaves=40, subsample=0.8, colsample_bytree=0.8, random_state=42),
    "cat": CatBoostRegressor(iterations=400, depth=6, learning_rate=0.05, verbose=0, random_state=42),
    "ridge": Ridge(alpha=1.0),
    "lasso": Lasso(alpha=0.01),
    "enet": ElasticNet(alpha=0.01, l1_ratio=0.5),
    "extr": ExtraTreesRegressor(n_estimators=300, max_depth=12, random_state=42)
}

oof_preds = {name: np.zeros(X.shape[0]) for name in base_models}
test_preds = {name: np.zeros(X_test.shape[0]) for name in base_models}

print("🚀 Training models...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X, bins)):
    print(f"🔁 Fold {fold + 1}")
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    for name, model in base_models.items():
        model.fit(X_tr, y_tr)
        oof_preds[name][val_idx] = model.predict(X_val)
        test_preds[name] += model.predict(X_test) / kf.n_splits


🚀 Training models...
🔁 Fold 1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000388 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2365
[LightGBM] [Info] Number of data points in the train set: 3992, number of used features: 19
[LightGBM] [Info] Start training from score 8.335701
🔁 Fold 2
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000434 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2363
[LightGBM] [Info] Number of data points in the train set: 3992, number of used features: 19
[LightGBM] [Info] Start training from score 8.336552
🔁 Fold 3
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000500 seconds.
You can set `force_row_wise=true` to r

## 📈 🔗 Stacking Meta-Model

Use a simple model (RidgeCV) to combine the outputs of all base models. This helps reduce individual model weaknesses.

In [47]:
# 📊 # 📊 Stack predictions using RidgeCV
oof_stack = np.column_stack([oof_preds[m] for m in base_models])
test_stack = np.column_stack([test_preds[m] for m in base_models])
meta_model = RidgeCV()
meta_model.fit(oof_stack, y)
final_oof = meta_model.predict(oof_stack)
final_test = meta_model.predict(test_stack)

# 🎯 Inverse transform
y_true = np.expm1(y)
y_pred = np.expm1(final_oof)
y_test = np.expm1(final_test)

## 📈 Model Evaluation
Evaluate model performance using R² and RMSE.

In [48]:
# 📈 Final Performance
print("\n✅ Pro-Level Stacked R²:", round(r2_score(y_true, y_pred), 4))
# Calculate RMSE by taking the square root of MSE
print("✅ Pro-Level Stacked RMSE:", round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)) # Changed line


✅ Pro-Level Stacked R²: 0.5364
✅ Pro-Level Stacked RMSE: 2915.38


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 💾 Submission Generation
Save and download final predictions in submission format.

In [49]:
# 📁 Save submission
submission = sample.copy()
submission['Item_Store_Returns'] = y_test
submission.to_csv("pro_ensemble_submission.csv", index=False)
files.download("pro_ensemble_submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# ✅ Improved GPT-4 Prediction Cell for DSN Clinical Challenge

!pip install --upgrade openai pandas scikit-learn

from google.colab import files
uploaded = files.upload()  # Upload train.csv and test.csv

import pandas as pd, re, time, random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import openai

# ✅ Replace with your real API key
openai.api_key = "sk-..."  # Insert your API key here

# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

def normalize(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

train['Prompt_norm'] = train['Prompt'].apply(normalize)
train['Clinician_norm'] = train['Clinician'].apply(normalize)
test['Prompt_norm'] = test['Prompt'].apply(normalize)

# Experience bucket
def bucket_experience(years):
    try:
        y = int(years)
        if y <= 3: return "junior"
        elif y <= 10: return "mid"
        else: return "senior"
    except: return "unknown"

train['ExperienceLevel'] = train['Years of Experience'].apply(bucket_experience)
test['ExperienceLevel'] = test['Years of Experience'].apply(bucket_experience)

vectorizer = TfidfVectorizer().fit(train['Prompt_norm'])
X_train = vectorizer.transform(train['Prompt_norm'])

def get_examples(prompt_norm, panel, level, exp, k=4):
    filt = train[(train['Clinical Panel'] == panel) & (train['Health level'] == level) & (train['ExperienceLevel'] == exp)]
    if len(filt) < k:
        filt = train
    X_filt = vectorizer.transform(filt['Prompt_norm'])
    sims = cosine_similarity(vectorizer.transform([prompt_norm]), X_filt)[0]
    return filt.iloc[sims.argsort()[-k:][::-1]]

def build_prompt(prompt_norm, panel, level, exp):
    examples = get_examples(prompt_norm, panel, level, exp)
    parts = []
    for _, row in examples.iterrows():
        response = row['Clinician_norm']
        if not response.startswith("summary"):
            response = "summary: " + response
        parts.append(f"Prompt: {row['Prompt_norm']}
Clinician:
{response}
")
    parts.append(f"Prompt: {prompt_norm}
Clinician:")
    return "\n".join(parts)

def postprocess(text):
    text = normalize(text)
    if not text.startswith("summary"):
        text = "summary: " + text
    if "plan" not in text:
        text += " plan: monitor vitals and refer if condition worsens"
    return text

def query_gpt(prompt):
    try:
        res = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a Kenyan clinical officer. Reply with: summary, likely diagnosis, plan."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=300
        )
        content = res.choices[0].message["content"]
        clean = postprocess(content)
        if len(clean.split()) < 12:
            raise ValueError("Too short")
        return clean
    except Exception as e:
        print("⚠️ Fallback used:", e)
        return normalize(train['Clinician'].sample(1).values[0])

results = []
for i, row in test.iterrows():
    print(f"[{i+1}/{len(test)}] {row['Master_Index']}")
    prompt = build_prompt(row['Prompt_norm'], row['Clinical Panel'], row['Health level'], row['ExperienceLevel'])
    pred = query_gpt(prompt)
    results.append({"Master_Index": row['Master_Index'], "Clinician": pred})
    time.sleep(1.1)

submission = pd.DataFrame(results)
submission.to_csv("submission.csv", index=False)
files.download("submission.csv")
print("✅ Done — submission.csv is ready for Zindi.")
